In [1]:
import nltk
import random
import json
import os
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('vader_lexicon')

# Initialize NLP tools
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
sentiment_analyzer = SentimentIntensityAnalyzer()

# Load or initialize scenarios
def load_scenarios():
    if os.path.exists("scenarios.json"):
        with open("scenarios.json", "r") as f:
            return json.load(f)
    else:
        return [
            {
                "scenario": "You are in a meeting and someone interrupts you while you are speaking. How would you handle this?",
                "expected_keywords": ["calm", "assertive", "understand", "continue", "respect"]
            },
            {
                "scenario": "Your team missed a deadline and you need to inform your client. How do you approach this situation?",
                "expected_keywords": ["apologize", "plan", "resolve", "responsibility", "next steps"]
            },
            {
                "scenario": "You have a conflict with a colleague over project responsibilities. How do you resolve it?",
                "expected_keywords": ["communicate", "listen", "compromise", "understand", "solution"]
            }
        ]

scenarios = load_scenarios()

def save_scenarios():
    with open("scenarios.json", "w") as f:
        json.dump(scenarios, f, indent=4)

def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum() and word not in stop_words]
    return tokens

def evaluate_response(user_response, expected_keywords):
    user_tokens = preprocess_text(user_response)
    score = sum(1 for word in expected_keywords if word in user_tokens)

    # TF-IDF Vectorization and Cosine Similarity
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([' '.join(user_tokens), ' '.join(expected_keywords)])
    similarity = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix[1:2])[0][0]

    return score, len(expected_keywords), similarity

def provide_feedback(score, max_score, similarity, sentiment_score):
    feedback = "Feedback: "
    if score == max_score:
        feedback += "Excellent response! You covered all the key points.\n"
    elif score >= max_score / 2:
        feedback += "Good job! You covered some key points, but there is room for improvement.\n"
    else:
        feedback += "You may need to work on this area. Try to include more relevant points in your response.\n"

    feedback += f"Similarity to ideal response: {similarity:.2f}\n"
    feedback += f"Sentiment Analysis: {'Positive' if sentiment_score > 0 else 'Negative' if sentiment_score < 0 else 'Neutral'}\n"

    if sentiment_score < 0:
        feedback += "Consider rephrasing your response to maintain a positive tone."

    return feedback

def add_scenario():
    scenario_text = input("Enter the scenario: ")
    expected_keywords = input("Enter the expected keywords (comma-separated): ").split(",")
    new_scenario = {
        "scenario": scenario_text,
        "expected_keywords": [word.strip() for word in expected_keywords]
    }
    scenarios.append(new_scenario)
    save_scenarios()
    print("Scenario added successfully!")

def main():
    print("Welcome to the Soft Skills Enhancement Application!\n")

    while True:
        print("1. Practice a scenario")
        print("2. Add a new scenario")
        print("3. Exit")
        choice = input("Choose an option: ")

        if choice == "1":
            scenario = random.choice(scenarios)
            print("\nScenario:", scenario["scenario"])
            user_response = input("\nYour Response: ")

            score, max_score, similarity = evaluate_response(user_response, scenario["expected_keywords"])
            sentiment_score = sentiment_analyzer.polarity_scores(user_response)["compound"]
            feedback = provide_feedback(score, max_score, similarity, sentiment_score)

            print("\n" + feedback)
        elif choice == "2":
            add_scenario()
        elif choice == "3":
            break
        else:
            print("Invalid option. Please try again.")

if __name__ == "__main__":
    main()


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


Welcome to the Soft Skills Enhancement Application!

1. Practice a scenario
2. Add a new scenario
3. Exit
Choose an option: 1

Scenario: You are in a meeting and someone interrupts you while you are speaking. How would you handle this?

Your Response: 1

Feedback: You may need to work on this area. Try to include more relevant points in your response.
Similarity to ideal response: 0.00
Sentiment Analysis: Neutral

1. Practice a scenario
2. Add a new scenario
3. Exit
Choose an option: 2
Enter the scenario: 3
Enter the expected keywords (comma-separated): the
Scenario added successfully!
1. Practice a scenario
2. Add a new scenario
3. Exit
Choose an option: 3
